# Script to evaluate survey responses using LLM



# INITIALIZE questions_list
# Extract questions from the questions.docx file
questions_file = "questions.docx"

if os.path.exists(questions_file):
    doc = Document(questions_file)
    questions_list = []
    
    # Extract text from each paragraph in the document
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        # Only add non-empty lines as questions
        if text:
            questions_list.append(text)
    
    print(f"Loaded {len(questions_list)} questions from {questions_file}")
    print("First 3 questions:")
    for i, q in enumerate(questions_list[:3], 1):
        print(f"  {i}. {q[:100]}{'...' if len(q) > 100 else ''}")
else:
    print(f"Warning: {questions_file} not found. Using sample questions.")
    questions_list = [
        "What is your opinion on the topic?",
        "How would you describe your experience?",
        "What suggestions do you have for improvement?"
    ]

print(f"\nTotal questions: {len(questions_list)}")

In [ ]:
# INITIALIZE spreadsheet_file
import openpyxl
from openpyxl import Workbook, load_workbook
from docx import Document
import os

# Create or load the output spreadsheet
output_file = "question_grader_output.xlsx"

if os.path.exists(output_file):
    spreadsheet_file = load_workbook(output_file)
    sheet = spreadsheet_file.active
    print(f"Loaded existing spreadsheet: {output_file}")
else:
    spreadsheet_file = Workbook()
    sheet = spreadsheet_file.active
    sheet.title = "Graded Survey"
    print(f"Created new spreadsheet: {output_file}")

print(f"Active sheet: {sheet.title}")

In [ ]:
#INITIALIZE llm_api_connection

In [ ]:
# Copy survey_data (rows 3 to n) from survey_spreadsheet to questions_spreadsheet
# Specify the survey file that contains the original responses
survey_file = "survey_data.xlsx"  # Change this to your actual survey file

if os.path.exists(survey_file):
    # Load the survey spreadsheet
    survey_workbook = load_workbook(survey_file)
    survey_sheet = survey_workbook.active
    
    # Get the dimensions of the survey data
    max_row = survey_sheet.max_row
    max_col = survey_sheet.max_column
    
    print(f"Survey file: {survey_file}")
    print(f"Survey sheet: {survey_sheet.title}")
    print(f"Total rows in survey: {max_row}")
    print(f"Copying rows 3 to {max_row}...")
    
    # Copy data from row 3 onwards to the output spreadsheet
    # Starting at row 3 in the destination as well (rows 1-2 reserved for questions and LLM answers)
    rows_copied = 0
    for row_idx in range(3, max_row + 1):
        for col_idx in range(1, max_col + 1):
            # Get value from survey sheet
            cell_value = survey_sheet.cell(row=row_idx, column=col_idx).value
            # Write to output sheet at the same position
            sheet.cell(row=row_idx, column=col_idx, value=cell_value)
        rows_copied += 1
    
    print(f"Successfully copied {rows_copied} rows of survey data")
    
    # Save the spreadsheet
    spreadsheet_file.save(output_file)
    print(f"Saved spreadsheet to {output_file}")
    
else:
    print(f"Warning: Survey file '{survey_file}' not found.")
    print("Please update the 'survey_file' variable with the correct filename.")
    print("For now, creating empty spreadsheet with structure ready for data.")

# Step 1: Populate first row with questions




In [ ]:
#FUNCTION populate_questions():
#    FOR each question IN questions_list:
#        column_index = get_next_available_column()
#        write_to_cell(row=1, col=column_index, value=question)
#    END FOR
#END FUNCTION

# Step 2: Generate LLM answers for each question (row 2)




In [ ]:
def generate_llm_answers():
    """Generate LLM answers for each question and write to row 2"""
    
    # Get all values from row 1 (questions)
    questions_row = worksheet.row_values(1)
    
    # Iterate through each question
    for col_index, question in enumerate(questions_row, start=1):
        if not question:  # Skip empty cells
            continue
            
        print(f"Generating answer for question in column {col_index}: {question[:50]}...")
        
        # Create prompt for the LLM
        prompt = f"Answer the following question: {question}"
        
        # Call the LLM API to get the answer
        message = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1024,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        # Extract the answer from the response
        llm_answer = message.content[0].text
        
        # Write LLM answer to row 2 in the same column
        worksheet.update_cell(2, col_index, llm_answer)
        
        print(f"✓ Answer written to row 2, column {col_index}")
    
    print("All LLM answers generated successfully!")

# Uncomment to run:
# generate_llm_answers()

# Step 3: Create evaluation columns




In [ ]:
print("Question grader setup complete.")

: 

In [ ]:
# FUNCTION create_evaluation_columns():
#     question_count = count_columns_with_questions()
    
#     FOR i = 1 TO question_count:
#         # Insert new column after each question column
#         question_col = get_column_index_for_question(i)
#         evaluation_col = question_col + 1
        
#         insert_column(at=evaluation_col)
        
#         # Add header for evaluation column
#         evaluation_header = "Q" + i + "_evaluation"
#         write_to_cell(row=1, col=evaluation_col, value=evaluation_header)
#     END FOR
# END FUNCTION

# Step 4: Evaluate each human answer



In [ ]:
def evaluate_human_answers(worksheet, anthropic_client):
    """
    Evaluate each human answer against the LLM's known answer.
    
    Args:
        worksheet: The worksheet object containing questions and answers
        anthropic_client: Initialized Anthropic client for API calls
    """
    # Get all values from the worksheet
    all_values = worksheet.get_all_values()
    
    if len(all_values) < 3:
        print("Not enough rows to evaluate (need at least 3 rows)")
        return
    
    headers = all_values[0]
    llm_answers = all_values[1]
    
    # Find question columns (columns without "_evaluation" suffix)
    question_columns = []
    for idx, header in enumerate(headers):
        if header and not header.endswith('_evaluation'):
            question_columns.append(idx)
    
    print(f"Found {len(question_columns)} questions to evaluate")
    
    # Iterate through all human responses (rows 3 onwards)
    for row_idx in range(2, len(all_values)):
        row_number = row_idx + 1  # 1-indexed for spreadsheet
        human_responses = all_values[row_idx]
        
        print(f"Evaluating row {row_number}...")
        
        # Evaluate each question for this human
        for question_num, question_col in enumerate(question_columns, start=1):
            question_text = headers[question_col]
            known_answer = llm_answers[question_col]
            
            # Get human answer
            if question_col < len(human_responses):
                human_answer = human_responses[question_col]
            else:
                human_answer = ""
            
            # Skip if human answer is empty
            if not human_answer or human_answer.strip() == "":
                print(f"  Q{question_num}: Skipping (no answer)")
                continue
            
            # Construct evaluation prompt
            evaluation_prompt = f"""Question: {question_text}
Known correct answer: {known_answer}
Human answer to evaluate: {human_answer}

Please evaluate the human answer compared to the known answer.
Provide a review including:
- Accuracy assessment
- Key points covered or missed
- Overall quality rating"""
            
            try:
                # Get LLM evaluation using Anthropic API
                message = anthropic_client.messages.create(
                    model="claude-3-5-sonnet-20241022",
                    max_tokens=1024,
                    messages=[
                        {"role": "user", "content": evaluation_prompt}
                    ]
                )
                
                evaluation_result = message.content[0].text
                
                # Write evaluation to the column right after the question column
                evaluation_col = question_col + 1
                # Convert to A1 notation (1-indexed)
                cell_address = f"{chr(65 + evaluation_col)}{row_number}"
                worksheet.update(cell_address, evaluation_result)
                
                print(f"  Q{question_num}: Evaluated and written to {cell_address}")
                
            except Exception as e:
                error_message = f"Error evaluating: {str(e)}"
                print(f"  Q{question_num}: {error_message}")
                # Write error to evaluation column
                evaluation_col = question_col + 1
                cell_address = f"{chr(65 + evaluation_col)}{row_number}"
                worksheet.update(cell_address, error_message)
    
    print("Human answer evaluation complete!")

# Main execution


In [ ]:
# FUNCTION main():
#     # Step 1: Set up questions
#     populate_questions()
    
#     # Step 2: Get LLM answers
#     generate_llm_answers()
    
#     # Step 3: Create evaluation columns
#     create_evaluation_columns()
    
#     # Step 4: Evaluate all human answers
#     evaluate_human_answers()
    
#     # Save spreadsheet
#     save_spreadsheet()
    
#     PRINT "Evaluation complete!"
# END FUNCTION

# # Run the script
# main()